# Convert code from Python to C#

In [1]:
# imports

import os
import io
import sys
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai as genai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer

In [2]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [3]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"
GEMINI_MODEL = "gemini-2.5-flash"

code_qwen = "Qwen/CodeQwen1.5-7B-Chat"
CODE_QWEN_URL = "https://h1vdol7jxhje3mpn.us-east-1.aws.endpoints.huggingface.cloud"

In [4]:
system_message = "You are an assistant that reimplements Python code in high performance C# for an Windows 10. "
system_message += "Respond only with C# code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C# response needs to produce an identical output in the fastest possible time. Keep implementations of random number generators identical so that results match exactly."
system_message += "I will use the code to write a file in .csx script, so the Main method is not required — you just write the code at the top level."

In [5]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C# with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C# code; do not explain your work other than a few comments. Main method is not required — you just write the code at the top level. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to using all necessary C# packages.\n\n"
    user_prompt += python
    return user_prompt

In [6]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [7]:
def messages_for_gemini(python):
    return [
        {"role": "user", "parts": system_message},
        {"role": "user", "parts": user_prompt_for(python)}
    ]

In [8]:
# write to a file called optimized.csx

def write_output(cpp):
    code = cpp.replace("```csharp","").replace("```","")
    with open("optimized.csx", "w") as f:
        f.write(code)

In [9]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [10]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace("```csharp","").replace("```","")

In [11]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace("```csharp","").replace("```","")

In [12]:
def stream_gemini(python):
    try:
        reply = ""
        model = genai.GenerativeModel(GEMINI_MODEL)
        result = model.generate_content(messages_for_gemini(python), stream=True)
        for chunk in result:
            reply += chunk.text
            yield reply.replace("```csharp","").replace("```","")
    except Exception as e:
        print(f"Error: {e}")

In [13]:
def execute_python(code):
        try:
            output = io.StringIO()
            sys.stdout = output
            exec(code)
        finally:
            sys.stdout = sys.__stdout__
        return output.getvalue()

In [14]:
def execute_csharp(code):
        write_output(code)
        try:
            result = subprocess.run(["dotnet-script", "optimized.csx"], capture_output=True, text=True)
            return result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [15]:
def stream_code_qwen(python):
    tokenizer = AutoTokenizer.from_pretrained(code_qwen)
    messages = messages_for(python)
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    client = InferenceClient(CODE_QWEN_URL, token=hf_token)
    stream = client.text_generation(text, stream=True, details=True, max_new_tokens=3000)
    result = ""
    for r in stream:
        result += r.token.text
        yield result    

In [16]:
css = """
.python {background-color: #306998;}
.csharp {background-color: #050;}
"""

In [17]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    elif model=="Gemini":
        result = stream_gemini(python)
    elif model=="CodeQwen":
        result = stream_code_qwen(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far    

In [18]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C#")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=pi, lines=10)
        csharp = gr.Textbox(label="C# code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "Gemini", "CodeQwen"], label="Select model", value="Gemini")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        csharp_run = gr.Button("Run C#")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        csharp_out = gr.TextArea(label="C# result:", elem_classes=["csharp"])

    convert.click(optimize, inputs=[python, model], outputs=[csharp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    csharp_run.click(execute_csharp, inputs=[csharp], outputs=[csharp_out])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
